# Flower Classification

In this

## Setup
### Imports

In [4]:
import tensorflow as tf
AUTO = tf.data.experimental.AUTOTUNE

In [45]:
filenames = ['\tfrecords-jpeg-192x192\train\00-192x192-798.tfrec']
dataset = tf.data.TFRecordDataset(img_paths, num_parallel_reads = AUTO)

### Filenames

In [67]:
path = r'.\tfrecords-jpeg-512x512'
TRAINING_FILENAMES = tf.io.gfile.glob(path + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(path + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(path + '/test/*.tfrec')

Loading the files

['.\\tfrecords-jpeg-512x512\\test\\00-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\01-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\02-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\03-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\04-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\05-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\06-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\07-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\08-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\09-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\10-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\11-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\12-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\13-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\14-512x512-462.tfrec',
 '.\\tfrecords-jpeg-512x512\\test\\15-512x512-452.tfrec']